In [17]:
from tensorflow import keras
import re 
import bert 
import tensorflow_hub as hub 
import tensorflow as tf 

In [8]:
avgBrightnessModel = keras.models.load_model('GIF models/avg_brightness_prediction_model.h5')
entropyModel = keras.models.load_model('GIF models/entropy_prediction_model.h5')
framerateModel = keras.models.load_model('GIF models/framerate_prediction_model.h5')

In [9]:
# Clean text 
# definition for function for removing html tags 
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

# definition for function for remove any punctuations and special characters
def preprocess_text(raw_tweaet):
    # Removing html tags
    tweet = remove_tags(raw_tweaet)
    # Removing html tags
    tweet = re.sub('[^a-zA-Z]', '', tweet)
    # Removing html tags
    tweet = re.sub(r"\s+[a-zA-Z]\s+", ' ', tweet)
    # Removing multiple spaces
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

In [12]:
# Create a tokenizer 
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# .numpy(): converts a tensor object into an numpy.ndarray
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

# Definition for function for convert tweet to ids 
def tokenize_tweets(text_tweets):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_tweets))

In [20]:
def format_text(raw_text):
    clean_text = preprocess_text(raw_text)
    tokenized_text = tokenize_tweets(clean_text)
    formatted_text = tf.ragged.constant([tokenized_text], dtype=tf.int32)
    return formatted_text

In [21]:
def predict_GIF_features(formatted_input):
    brightness = avgBrightnessModel.predict(formatted_input)
    entropy = entropyModel.predict(formatted_input)
    framerate = framerateModel.predict(formatted_input)
    summary = {'Average_Brightness': brightness, 'Entropy': entropy, 'Framerate': framerate}
    return summary 

In [22]:
# Demonstration 
result = predict_GIF_features(format_text("Happy to meet you at last, Yully."))

In [23]:
result

{'Average_Brightness': array([[0.08106247]], dtype=float32),
 'Entropy': array([[0.70854807]], dtype=float32),
 'Framerate': array([[1.6424409]], dtype=float32)}